# Библиотекид ля работы с CV

## Предобработка изображений с помощью cv2 и Pillow

Две самые популярные библиотеки для работы с приложениями — это `cv2` и `Pillow`.

- cv2. Производительная библиотека, написанная на C++, которая позволяет осуществлять множество операций над изображениями и видео. Она содержит много методов для их обработки, которые были популярны до появления нейросетей, то есть в эпоху так называемого классического Computer Vision.

- Pillow. Более молодая библиотека, написанная на языке C. Её основной фокус направлен на предобработку изображений и простоту.

In [2]:
import requests

import cv2
from PIL import Image
import numpy as np

from ml_dl_experiments import settings

url = "https://raw.githubusercontent.com/jigsawpieces/dog-api-images/main/eskimo/n02109961_10021.jpg"
response = requests.get(url, stream=True)

image_path: str = settings.SOURCE_PATH + "ml_dl/CNN/eskimo_dog.jpg"

with open(image_path, "wb") as f:
    f.write(response.content)

img_pil = Image.open(image_path)
img_cv2 = cv2.imread(image_path)

В отличие от Pillow, cv2 использует другой порядок каналов при загрузке — BGR, а не RGB. Поэтому при чтении необходимо приводить изображение в стандартный формат с помощью перестановки каналов cv2.COLOR_BGR2RGB в методе cv2.cvtColor

In [ ]:
img_cv2 = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB) # type:ignore BGR -> RGB 

In [5]:
# не выведет ничего при ~равенстве массивов, иначе - ошибка
np.testing.assert_allclose(img_cv2, img_pil, atol=1e-8)

Выполним масштабирование

In [8]:
shape = (224, 224)
img_pil = img_pil.resize(shape)
img_cv2 = cv2.resize(img_cv2, shape)

np.testing.assert_allclose(img_cv2, img_pil, atol=1e-8) 

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=1e-08

Mismatched elements: 94421 / 150528 (62.7%)
Max absolute difference among violations: 56
Max relative difference among violations: 3.
 ACTUAL: array([[[ 91, 116,  58],
        [ 88, 113,  56],
        [ 89, 114,  57],...
 DESIRED: array([[[ 90, 115,  58],
        [ 88, 113,  56],
        [ 89, 114,  57],...

Здесь уже видны различия — из-за разных алгоритмов масштабирования и их реализации мы не всегда можем получить одинаковые результаты, вызывая в общем схожие методы. Расхождения обычно небольшие, так что явных артефактов на изображении не будет. Но лучше всё-таки помнить про это, фиксировать один алгоритм ремасштабирования и в дальнейшем придерживаться его.

Нормализация:

In [9]:
img_pil = np.array(img_pil) / 255
img_cv2 = img_cv2 / 255

print(np.min(img_cv2), np.min(img_pil), np.max(img_cv2), np.max(img_pil)) 

0.0 0.0 0.9490196078431372 0.9529411764705882


In [13]:
img_pil.shape

(224, 224, 3)

Для изображения с shape=(224, 224, 3) в формате RGB, чтобы вычислить среднее и стандартное отклонение для каждого канала с помощью numpy, используем следующие подходы:

## Расчёт mean и std для каналов

```python
import numpy as np

# Пусть img — numpy-массив изображения с shape=(224, 224, 3)
mean = np.mean(img, axis=(0, 1))  # Среднее для каждого канала
std = np.std(img, axis=(0, 1))    # Стандартное отклонение для каждого канала
```

## Детали

- Аргумент axis=(0, 1) означает, что функция считается по высоте и ширине, отдельно для каждого канала (последнее измерение — RGB).
- В результате mean и std будут массивами из 3 элементов: [mean_R, mean_G, mean_B] и [std_R, std_G, std_B].

## Пример вывода

```python
print("Mean (R, G, B):", mean)
print("Std  (R, G, B):", std)
```

Для типичных задач машинного обучения mean и std используют для нормализации входных данных изображения.

При загрузке порядок размерностей H × W × C различается от того, какой требует PyTorch при использовании моделей — C × H × W.
Чтобы преобразовать изображения к формату Channels × Heigth × Width, можно воспользоваться стандартной перестановкой каналов из NumPy: методом transpose. Он применяется к массивам и принимает на вход индексы каналов в нужном нам порядке. 

In [20]:
print(f"Размеры до: {img_cv2.shape}")

img_cv2 = img_cv2.transpose(2, 0, 1)
print(f"Размеры после: {img_cv2.shape}")

Размеры до: (356, 356, 3)
Размеры после: (3, 356, 356)


Библиотека cv2 может быть предпочтительна в ситуациях, когда нужна более быстрая обработка.
cv2, написанная на C++, быстрее, чем Pillow.

In [18]:
# читаем картинку
img_cv2 = cv2.imread(image_path)
# переводим в RGB цвета
img_cv2 = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
# Масшатбируем
shape = (356, 356)
img_cv2 = cv2.resize(img_cv2, shape)
# Приводим к диапазону 0-1
img_cv2 = img_cv2 / 255
# Поканальная нормализация
# mean = np.mean(img_cv2, axis=(0, 1))
# std = np.std(img_cv2, axis=(0, 1))

mean = np.array([0.4, 0.42, 0.44])
std =np.array([0.2, 0.21, 0.22])

img_cv2 = (img_cv2 - mean) / std 

print(img_cv2[0,0,1].round(3))
mean, std

0.166


(array([0.4 , 0.42, 0.44]), array([0.2 , 0.21, 0.22]))

Чаще всего представленные библиотеки для задач DL используются как загрузчики данных — выбор конкретного фреймворка в таком сценарии использования не так критичен. 

Но когда вы хотите выполнять обработку видео, применять какие-то методы из классического компьютерного зрения и при этом иметь большую скорость обработки, стоит отдать предпочтение cv2.

## Библиотека  timm


Библиотека timm — Torch IMage Models, входящая в экосистему платформы huggingface. timm значительно упрощает загрузку, использование и настройку моделей под конкретные задачи CV.

In [1]:
import timm

total_models = len(timm.list_models())
print(f"Общее число моделей: {total_models}")
print(f"Первые 5 моделей семейства resnet: \n{timm.list_models(filter='resnet*')[:5]}")

Общее число моделей: 1279
Первые 5 моделей семейства resnet: 
['resnet10t', 'resnet14t', 'resnet18', 'resnet18d', 'resnet26']


Нобходимую модель нужно скачать и проинициализировать. Делается это с помощью метода `create_model(model_name=..)`:

In [3]:
model = timm.create_model(model_name="resnet18")
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

По умолчанию модели инициализируются случайными весами. Чтобы использовать предобученный вариант, укажем pretrained=True, а также визуализируем конфигурационный файл с помощью атрибута модели pretrained_cfg 

In [4]:
model = timm.create_model(
    model_name="resnet18",
    pretrained=True
)

model.pretrained_cfg

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

{'url': 'https://github.com/huggingface/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet18_a1_0-d63eafa0.pth',
 'hf_hub_id': 'timm/resnet18.a1_in1k',
 'architecture': 'resnet18',
 'tag': 'a1_in1k',
 'custom_load': False,
 'input_size': (3, 224, 224),
 'test_input_size': (3, 288, 288),
 'fixed_input_size': False,
 'interpolation': 'bicubic',
 'crop_pct': 0.95,
 'test_crop_pct': 1.0,
 'crop_mode': 'center',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'num_classes': 1000,
 'pool_size': (7, 7),
 'first_conv': 'conv1',
 'classifier': 'fc',
 'license': 'apache-2.0',
 'origin_url': 'https://github.com/huggingface/pytorch-image-models',
 'paper_ids': 'arXiv:2110.00476'}

Основные параметры, которые могут быть полезны:

- `tag` — как правило, содержит информацию об обучении модели. В частности, здесь a1_in1k — означает, что модель обучалась на датасете ImageNet с 1000 классами.
    
- `input_size` — размер изображений, использованный при тренировке.
    
- `fixed_input_size` — атрибут, указывающий на жёсткость ограничения входного размера изображений. В данном случае False означает, что ограничений нет.
    
- `mean и std` — коэффициенты для нормализации изображений, использованные при тренировке.

Для загрузки pretrained модели с помощью timm из локального файла нужно сделать следующее:

## Загрузка модели timm с локальной pretrained весовой:

1. Создайте модель с `pretrained=False`, чтобы не загружать веса из интернета.
2. Загрузите веса из локального файла (обычно `.pth` или `.bin`) с помощью `torch.load`.
3. Подгрузите веса в модель через `model.load_state_dict`.
4. Модель готова к использованию.

## Пример кода:

```python
import timm
import torch

# Создаем модель без загрузки pretrained весов из интернета
model = timm.create_model('имя_модели', pretrained=False)

# Путь к файлу с весами на локальном диске
weights_path = 'путь/до/файла/model_weights.pth'

# Загружаем веса
state_dict = torch.load(weights_path, map_location='cpu')

# Если в state_dict есть ключ 'model' или 'state_dict', нужно извлечь
# Пример для common case:
if 'state_dict' in state_dict:
    state_dict = state_dict['state_dict']

# Загружаем веса в модель
model.load_state_dict(state_dict)

# Модель готова для использования
model.eval()
```

### Особенности
- Убедитесь, что название модели в create_model совпадает с той, для которой предназначены веса.
- Если файл весов был сохранен с использованием `DataParallel` (с префиксом 'module.' в ключах), возможно потребуется подправить ключи перед загрузкой.

Так вы можете передать путь к файлу с pretrained весами и загрузить их локально без подключения к интернету.

Оценим инференс одной модели. maxvit_large_tf_384
1. Загрузим картинку и модель
2. Масштабируем в нужный формат
3. Нормализуем в диапазон от 0 до 1, затем в формат, необходимый для тренировки конкретной модели
4. Вывести результирующую размерность изображения

In [10]:
import timm
import torch.nn as nn
import cv2
import numpy as np

from ml_dl_experiments import settings
# Загрузим изображение
image_path: str = settings.SOURCE_PATH + "ml_dl/CNN/eskimo_dog.jpg"

img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img / 255

In [ ]:
# Загрузим модель, выведем конфиги

model = timm.create_model(
    model_name="maxvit_large_tf_384",
    pretrained=True
)

model.pretrained_cfg


model.safetensors:   0%|          | 0.00/849M [00:00<?, ?B/s]

{'url': '',
 'hf_hub_id': 'timm/maxvit_large_tf_384.in1k',
 'architecture': 'maxvit_large_tf_384',
 'tag': 'in1k',
 'custom_load': False,
 'input_size': (3, 384, 384),
 'fixed_input_size': True,
 'interpolation': 'bicubic',
 'crop_pct': 1.0,
 'crop_mode': 'squash',
 'mean': (0.5, 0.5, 0.5),
 'std': (0.5, 0.5, 0.5),
 'num_classes': 1000,
 'pool_size': (12, 12),
 'first_conv': 'stem.conv1',
 'classifier': 'head.fc',
 'license': 'apache-2.0'}

In [5]:
mean, std = np.array(model.pretrained_cfg['mean']), np.array(model.pretrained_cfg['std'])
shape = (384, 384)

In [11]:
# Изменим масштаб и нормализуем под модель
img = cv2.resize(img, shape)

img = (img - mean) / std
img.shape

(384, 384, 3)

In [19]:
import torch
from torchvision.transforms import ToTensor
tensor = ToTensor()
image_tensor = tensor(img)
image_tensor = image_tensor.to(torch.float32)
model.eval()
with torch.no_grad():
    out = model(image_tensor.unsqueeze(0))

result = out.softmax(dim=1)
top5_probabilities, top5_class_indices = torch.topk(result * 100, k=5)
top5_probabilities, top5_class_indices

(tensor([[65.1152, 23.5729,  0.7286,  0.1099,  0.1031]]),
 tensor([[250, 248, 249, 174, 270]]))

Ещё одна важная часть возможностей библиотеки связана с инициализацией самой модели.

Большая часть моделей обучались на задачу классификации и построены по упрощённому принципу:

- последовательные слои конволюций,
    
- агрегирование значений карт признаков,

- линейный классификатор.

Для конечной задачи не всегда интересен результирующий классификатор — больше наиболее важны признаки модели (тензора до линейного слоя). 

Производить `feature-extraction` в `timm` легко — достаточно указать `features_only=True`. 

```py
model = timm.create_model("efficientnet_b0",
                          pretrained=True,
                          features_only=True)

x = torch.randn(1, 3, 224, 224)
out = model(x)
print([x.shape for x in out])
```

```py
[torch.Size([1, 16, 112, 112]), 
 torch.Size([1, 24, 56, 56]), 
 torch.Size([1, 40, 28, 28]), 
 torch.Size([1, 112, 14, 14]), 
 torch.Size([1, 320, 7, 7])]
```

Получаем разноуровневые признаки из модели — какие-то будут акцентироваться на крупных деталях, какие-то на мелких. Часто использование признаков разных уровней помогает в решении задачи или улучшении качества существующей.

Например, это эффективно в задаче детекции, когда мы хотим обнаруживать как локальные паттерны (ранние слои: цвета и градиенты), так и глобальные (части объектов: глаза, руки и т. д.). 

Под «уровнем» здесь подразумевается один из последовательных блоков. В частности, в `efficientnet_b0` это индекс группы `InvertedResidual` - блоков — таких в этой модели 5 штук.

Количеством извлекаемых признаков можно управлять с помощью параметра `out_indices`, в который передаётся список индексов, соответствующий блокам признаков. 

Для некоторых задач могут быть релевантны только признаки с самых глубоких слоёв — и для модели из примера выше мы могли бы извлечь, например, только 2 последних признака. 

Чтобы понять общее количество возвращаемых признаков для конкретной модели, можно сначала указать только `features_only`, а по количеству тензоров понять, какие индексы можно указывать.

```py
model = timm.create_model("efficientnet_b0",
                          pretrained=True,
                          features_only=True,
                          out_indices=[3, 4])

x = torch.randn(1, 3, 224, 224)
out = model(x)
print([x.shape for x in out])

[torch.Size([1, 112, 14, 14]), 
 torch.Size([1, 320, 7, 7])]
```

Без библиотеки timm нам пришлось бы вручную реализовывать поддержку каждой архитектуры на PyTorch, а timm берёт эту работу на себя и обеспечивает единый интерфейс для любых моделей.